# Part of Speech (POS) Tagging with Hidden Markov Models (HMMs)

In this lesson, we introduce **Hidden Markov Models (HMMs)**, which are used to model sequences (including time-series data).

**HMMs** have been successfully used on both supervised and unsupervised problems with sequence data in applications like speech recognition, bioinformatics, sign language recognition, and gesture recognition.

This lesson is closely related to the project at the end of this course, where you will use a **Hidden Markov Model (HMM)** to tag parts of speech in English sentences. This is a common pre-processing task in natural language processing.

## What are parts of speech

Parts of speech are labels typically applied to words in a sentence. There are many potential parts of speech for most languages, but for this, we will focus on only three:

<img src='img_40.png' width=700 align='center'>

Identifything these in sentences is known as **part-of-speech tagging**:

<img src='img_41.png' width=700 align='center'>

where "mary" is a noun, "had" is a verb, "a" is a determinant, "little" is an adjective, and "lamb" is a noun

## Using a LookUp Table to Assign POS - Rudimentary Model

Say you have two sentences for training data:
1. "Mary saw Jane" -> {"Mary: "noun", "saw": "verb", "Jane": "noun"}
2. "Jane saw Will" -> {"Jane: "noun", "saw": "verb", "Will": "noun"}

So, we can create a table:

| | Noun | Verb|
|:-|:-:|:-:|
|Mary|1|0|
|saw|0|2|
|Jane|2|0|
|Will|1|0|

So now, if we have a sentence "Mary saw Will", we can use the lookup table and note:
* Mary is a noun 100% of the time (in training data)
* Saw is a verb 100% of the time (in training data)
* Will is a noun 100% of the time (in training data)

So we can tag this sentence as such:
* "Mary saw Will" -> {"Mary: "noun", "saw": "verb", "Will": "noun"}


<img src='img_42.png' width=700 align='center'>

So, that worked! However, we can't reasonably expect the LookUp Table method to work all, or maybe most, of the time

Let's use a more complex sentence series now as an example:

Say you have three sentences for training data:
1. "Mary will see Jane" -> {"Mary": "noun", "will": "modal", "see": "verb", "Jane": "noun"}
2. "Will will see Mary" -> {"Will": "noun", "will": "modal", "see": "verb", "Mary": "noun"}
3. "Jane will see Will" -> {"Jane": "noun", "will": "modal", "see": "verb", "Will": "noun"}

So, we can create a table:

| | Noun | Verb| Modal|
|:-|:-:|:-:|:-:|
|Mary|3|0|0|
|see|0|3|0|
|Jane|2|0|0|
|Will|2|0|3|


Now, lets take a new sentence, "Mary will see Will", and tag it based on the LookUp Table:
* Mary is a noun 100% of the time -> Mary: noun
* will is a modal $\frac{3}{5}$ and a noun $\frac{2}{5}$ of the time -> will: modal
* see is a verb 100% of the time-> see:verb

So we can tag this sentence as such:
* "Mary will see Will" -> {"Mary: "noun", "see": "verb", "will": "modal"}

But this is a problem, because we know Will is a noun and not a modal! This LookUp Table works by assigning the most prevalent POS Tag and isn't aware of context!

<img src='img_43.png' width=700 align='center'>

### Bi-Gram LookUp Table

So, how can the LookUp table take advantage of context? We can assign frequencies instead to groups of words, namely n-grams!

| | Noun-Modal (N-M)| Modal-Verb (M-V)| Verb-Noun (V-N)|
|:-|:-:|:-:|:-:|
|mary-will|1|0|0|
|will-see|0|3|0|
|see-jane|0|0|1|
|will-will|1|0|0|
|see-mary|0|0|1|
|jane-will|1|0|0|
|see-will|0|0|1|

So now, lets take the same sentence as before, "Mary will see Will", and tag it based on the bi-gram LookUp Table:
* mary will is a noun-modal 100% of the time -> Mary: noun, will: modal
* will see is a modal-verb 100% of the time -> will: modal, see: verb
* see will is a verb-noun 100% of the time -> see: verb, will: noun

So: "Mary will see Will" -> Mary:noun, will:modal, see:verb, Will: noun

<img src='img_44.png' width=700 align='center'>

### But will bi-grams always work?

Let's change our training data a bit:
1. "Mary Jane can see Will." -> {"Mary": "noun", "Jane": "noun", "can": ,"see": "verb", "Will": "noun"}
2. "Spot will see Mary" -> {"Will": "noun", "will": "modal", "see": "verb", "Mary": "noun"}
3. "Will Jane spot Mary?"
4. "Mary will pat Spot." -> {"Jane": "noun", "will": "modal", "see": "verb", "Will": "noun"}

And let's tag the sentence: "Jane will spot Will."

| | Noun-Modal (N-M)| Modal-Verb (M-V)| Verb-Noun (V-N)|etc|
|:-|:-:|:-:|:-:|:-:|
|mary-jane|0|0|0|1|
|jane-can|0|0|0|1|
|can-see|0|0|0|1|
|see-will|0|0|0|1|
|spot-will|1|0|0|0|
|will-see|0|1|0|0|
|see-mary|0|0|1|0|
|will-jane|0|0|1|0|
|jane-spot|0|0|0|1|
|spot-mary|0|0|1|0|
|mary-will|1|0|0|0|
|will-pat|0|1|0|0|
|pat-spot|0|0|1|0|

But wait, the bi-gram "jane-will" does not appear in the training data... so we have no way to fill in the extra information! So instead, we'll introduce **hidden Markov models**

## Hidden Markov Models (HMM)

The idea of HMM is the following: say that a way of tagging the sentence "Jane will spot Will" is noun-modal-verb-noun, so we'll calculate a probability associated with this tagging. So first, well ask "how likely is it that a noun is followed by a modal and a modal is followed by a verb and a verb is followed by a noun". These are called the **transition probabilities**. Now, the second set of probabilities we need to calculate are these: "what is the probability that a noun will be the word *Jane* and that a modal will be the word *will*, etc." These also need to be relatively high for our tagging to be likely. These are called the **Emission Probabilities**

<img src='img_45.png' width=700 align="center">

So if we have the following sentences, we can calculate the emission probabilities:
1. Mary Jane can see Will
2. Spot will see Mary
3. Will Jane spot Mary
4. Mary will pat Spot

We can create a LookUp Table as such:

||Noun|Modal|Verb|
|:-|:-:|:-:|:-:|
|Mary|4|0|0|
|Jane|2|0|0|
|Will|1|3|0|
|Spot|2|0|1|
|Can|0|1|0|
|See|0|0|2|
|Pat|0|0|1|

To find the probabilities (frequencies), we sum the columns' values, then we divide the values in each row by that total
* Noun=9
* Modal=4
* Verb=4

||Noun|Modal|Verb|
|:-|:-:|:-:|:-:|
|Mary|$\frac{4}{9}$|0|0|
|Jane|$\frac{2}{9}$|0|0|
|Will|$\frac{1}{9}$|$\frac{3}{4}$|0|
|Spot|$\frac{2}{9}$|0|$\frac{1}{4}$|
|Can|0|$\frac{1}{4}$|0|
|See|0|0|$\frac{2}{4}$|
|Pat|0|0|$\frac{1}{4}$|

<img src='img_46.png' width=700 align="center">

Graphically:

<img src='img_47.png' width=700 align="center">


Now, let's calculate the transition probabilities:
1. Add starting `<s>` and ending `<e>` tags to each sentence and we'll treat these tags as parts of speech as well
2. Make a table of counts!

**Tag Start and End:**
1. \<s>Mary Jane can see Will\<e>
2. \<s>Spot will see Mary\<e>
3. \<s>Will Jane spot Mary\<e>
4. \<s>Mary will pat Spot\<e>

**Make a table of counts:**

|Preceding/Succeeding|Noun|Modal|Verb|\<e>|
|:-|:-:|:-:|:-:|:-:|
|\<s>|3|1|0|0|
|Noun|1|3|1|4|
|Modal|1|0|3|0|
|Verb|4|0|0|0|

Where the values going down the left are the POS tag of the preceding term and the values going across are the succeeding POS tag of the term. So, noun modal happens 3 times, just like starting tag and noun happen 3 times

<img src='img_48.png' width=700 align="center">

Now, to find the probabilities (frequencies), the transition probabilities (matrix), we divide the sum of occurences in the **row**

|Preceding/Succeeding|Noun|Modal|Verb|\<e>|
|:-|:-:|:-:|:-:|:-:|
|\<s>|$\frac{3}{4}$|$\frac{1}{4}$|0|0|
|Noun|$\frac{1}{9}$|$\frac{3}{9}$|$\frac{1}{9}$|$\frac{4}{9}$|
|Modal|$\frac{1}{4}$|0|$\frac{3}{4}$|0|
|Verb|$\frac{4}{4}$|0|0|0|

So, if we take the **modal row**, the probability that a noun follows a modal is $\frac{1}{4}$ and is $\frac{3}{4}$ it's a verb, and is $0$ that it's a modal or the end of a sentence

Now, graphically, we have:

<img src='img_49.png' width=700 align="center">

As a final step, we combine the previous two steps (emission probabilities and transition probabilities) to get the HMM:
1. We have our words which are the observations
> * They are called the observations because they are the things we observe when we read the sentences
2. The parts of speech are called the *hidden states*
> * They are the ones we don't know and we have to infer based on the words
3. The transition probabilities (among the hidden states)
4. The emission probabilities (between the hidden states)

<img src='img_50.png' width=700 align="center">
